In [31]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch.nn as nn
from torchvision import models

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [32]:
import torch.nn as nn
import torch
import torch.optim as optim
class AlexNet(nn.Module):
    def __init__(self, num_classes=1000, init_weights=False):   
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(  #打包
            nn.Conv2d(3, 48, kernel_size=11, stride=4, padding=2),  # input[3, 224, 224]  output[48, 55, 55] 自动舍去小数点后
            nn.ReLU(inplace=True), #inplace 可以载入更大模型
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[48, 27, 27] kernel_num为原论文一半
            nn.Conv2d(48, 128, kernel_size=5, padding=2),           # output[128, 27, 27]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 13, 13]
            nn.Conv2d(128, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 192, kernel_size=3, padding=1),          # output[192, 13, 13]
            nn.ReLU(inplace=True),
            nn.Conv2d(192, 128, kernel_size=3, padding=1),          # output[128, 13, 13]
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),                  # output[128, 6, 6]
        )
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            #全链接
            nn.Linear(128 * 6 * 6, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(2048, 2048),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.5),
            nn.Linear(2048, num_classes),
        )
        if init_weights:
            self._initialize_weights()

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, start_dim=1) #展平   或者view()
        x = self.classifier(x)
        return x


In [33]:
import torch
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

#建立自己的dataset train_data
class MyDataset(Dataset):
    def __init__(self, image_path: list, image_class: list, transform=None):
        self.image_path = image_path
        self.image_class = image_class
        self.transform = transform

    def __len__(self):
        return len(self.image_path)

    def __getitem__(self, item):
        img = Image.open(self.image_path[item]).convert('RGB')
#         if img.mode != 'RGB':
#             raise ValueError("image: {} isn't RGB mode".format(self.image_path[item]))
        label = self.image_class[item]

        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [34]:
import os
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split
from torch.autograd import Variable

epoch_num=20
batch_size=32
lr=0.01
path_photo='../input/bitmoji-faces-gender-recognition/BitmojiDataset/trainimages'
path_label='../input/bitmoji-faces-gender-recognition/train.csv'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

# temp_name = os.listdir(path_photo)
df=pd.read_csv(path_label)
image_path=[]
label_path = []
for dir in os.listdir(path_photo):
#     print(dir)
    image_path.append(path_photo+'/'+str(dir))
    # 得到所有图片label
    if df[df.image_id == dir].iloc[0, 1]==-1:
        label_path.append(0)
    else:
        label_path.append(df[df.image_id == dir].iloc[0, 1])

image_path=np.array(image_path)
label_path=np.array(label_path)

train_path,val_path,train_label,val_label=train_test_split(image_path,label_path,test_size=0.2,random_state=1)
print(len(train_path),len(val_path))
train_data=MyDataset(train_path,train_label,transform)
val_data=MyDataset(val_path,val_label,transform)
train_loader = DataLoader(train_data, batch_size, shuffle=False)
val_loader = DataLoader(val_data, batch_size,shuffle=False)
print(train_loader)
print(val_loader)

2400 600


In [35]:
net=AlexNet()
if torch.cuda.is_available():
    net=net.cuda()
# 参数量
n_p=sum(x.numel() for x in net.parameters())
print(n_p)

16630440


In [36]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(), lr=lr) 

In [40]:
# 训练
import time
acc=0
n=0
print('start training!')
start=time.time()
for epoch in range(epoch_num):
    for img,label in train_loader:
        # 将数据移到GPU上
        if torch.cuda.is_available():
            img, label = img.cuda(), label.cuda()
        # 先将optimizer梯度先置为0
        optimizer.zero_grad()
        output=net(img)
        # 计算loss,图的终点处
        loss=criterion(output,label)
#         反向传播
        loss.backward()
        # 更新参数
        optimizer.step()
        prect =output.argmax(1)
        num_correct=(prect==label).sum()
        acc+=num_correct.item()
        n+=label.shape[0]
    print('epoch: {},  loss: {:.4} Acc: {:.6f}'.format(epoch,  loss.data.item(),acc / n))
start=time.time()-start
print(start)

print("train over!")
if not os.path.exists('./model_save'):
    os.mkdir('./model_save')
torch.save(net.state_dict,os.path.join('./model_save/','epoch_{}_.pth'.format(epoch+1)))
#     if (epoch%50 == 0) & (epoch != 0):
#         i = epoch/50
#         torch.save(model, 'AlexNet%03d.pth'% i)

start training!
epoch: 0,  loss: 0.1638 Acc: 0.972083
epoch: 1,  loss: 0.1434 Acc: 0.972708
epoch: 2,  loss: 0.1249 Acc: 0.973194
epoch: 3,  loss: 0.1094 Acc: 0.973646
epoch: 4,  loss: 0.09804 Acc: 0.974417
epoch: 5,  loss: 0.08551 Acc: 0.975625
epoch: 6,  loss: 0.07396 Acc: 0.976607
epoch: 7,  loss: 0.06413 Acc: 0.977448
epoch: 8,  loss: 0.05617 Acc: 0.978194
epoch: 9,  loss: 0.05011 Acc: 0.979125
epoch: 10,  loss: 0.04631 Acc: 0.979924
epoch: 11,  loss: 0.04219 Acc: 0.980625
epoch: 12,  loss: 0.03667 Acc: 0.981314
epoch: 13,  loss: 0.03326 Acc: 0.981935
epoch: 14,  loss: 0.02887 Acc: 0.982583
epoch: 15,  loss: 0.02465 Acc: 0.983177
epoch: 16,  loss: 0.02247 Acc: 0.983750
epoch: 17,  loss: 0.0198 Acc: 0.984306
epoch: 18,  loss: 0.01774 Acc: 0.984825
epoch: 19,  loss: 0.01626 Acc: 0.985313
261.1684060096741
train over!


In [41]:
import torch
# 模型评估
net.eval()
eval_loss=0
eval_acc=0
for img,label in val_loader:
    if torch.cuda.is_available():
        img=img.cuda()
        label=label.cuda()
    out=net(img)
    loss=criterion(out,label)
    eval_loss+=loss.item()*label.size(0)
    prect =out.argmax(1)
    num_correct=(prect==label).sum()
    eval_acc+=num_correct.item()
print('Test Loss: {:.6f}, Acc: {:.6f}'.format(eval_loss / (len(val_data)), eval_acc / (len(val_data))))

    
    


Test Loss: 0.024324, Acc: 0.995000


In [39]:
# 测试
import torch
import pandas as pd
testimg_path='../input/bitmoji-faces-gender-recognition/BitmojiDataset/testimages'
submission_path='../input/bitmoji-faces-gender-recognition/sample_submission.csv'

cnt=0
test_df=pd.read_csv(submission_path)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224))
])

# net.eval()
with torch.no_grad():
    parent_list=os.listdir(testimg_path)
    parent_list.sort()
#     print(parent_list)
    for img_name in parent_list:
        img_path = os.path.join(testimg_path, img_name)
        img = Image.open(img_path).convert('RGB')
        img = transform(img)
        if torch.cuda.is_available():
            img=img.cuda()
        img = img.unsqueeze(0)
        prect = int(net(img).argmax(1))
        if prect==0:
            prect=-1
#         print(prect)
        test_df['is_male'].iloc[cnt:cnt+1]=str(prect)
        cnt+=1
# test_df.iloc[:,1:2]
print(test_df)    
test_df.to_csv('./sample_submission.csv',index=False,header=True)
print('test_over!')
       

/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


      image_id is_male
0     3000.jpg      -1
1     3001.jpg      -1
2     3002.jpg      -1
3     3003.jpg       1
4     3004.jpg      -1
...        ...     ...
1079  4079.jpg      -1
1080  4080.jpg       1
1081  4081.jpg       1
1082  4082.jpg      -1
1083  4083.jpg       1

[1084 rows x 2 columns]
test_over!
